In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [6]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [7]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en_core_web_sm')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [8]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [11]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(500)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [12]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,ill,dreadfully,mary,surprise,hatter,longer,right,tell,bad,dish,...,pride,would,moral,heart,here,deep,improve,yes,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [14]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9799373040752352

Test set score: 0.8947368421052632


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [15]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 774) (3190,)
Training set score: 0.9423197492163009

Test set score: 0.9163533834586466


# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [16]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8874608150470219

Test set score: 0.8745300751879699


## Same model, new inputs
What if we feed the model a different novel by Jane Austen, like Emma? Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process Emma the same way we processed the other data, and combine it with the Alice data:

In [17]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [18]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [19]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [20]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [21]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.691648023862789


col_0,Austen,Carroll
row_0,,
Austen,1561,108
Carroll,719,294


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  



In addition to unique words let's add a count of punctuations, adjuctives (ADJ), verbs(VERB), and conjunctions(CONJ) to the word_counts data frame.

In [24]:
# Copy first
word_counts_0 = word_counts.copy()

In [94]:
text_sentence = word_counts_0.text_sentence
# punctuation mark count in each sentence
punc_count = []
for sent in text_sentence:
    punc = 0
    for token in sent:
        if token.pos_ == 'PUNCT':
            punc +=1
    punc_count.append(punc)

word_counts_0['punc_count'] = punc_count

# Adjuctive count in each sentence 
#adj_count = []
#for sent in text_sentence:
#    adj = 0
#    for token in sent:
 #       if token.pos_ == 'ADJ':
 #           adj +=1
#    adj_count.append(adj)

# add as a column
#word_counts_0['adj_count'] = adj_count

# verb count in each sentence
verb_count = []
for sent in text_sentence:
    verb = 0
    for token in sent:
        if token.pos_ == 'VERB':
            verb +=1
    verb_count.append(verb)
    
# add as a column
word_counts_0['verb_count'] = verb_count

# conjunction count 
conj_count = []
for sent in text_sentence:
    conj = 0
    for token in sent:
       if token.pos_ == 'CONJ':
            conj +=1
    conj_count.append(conj)

word_counts_0['conj_count'] = conj_count

In [103]:
#Confirm changes
word_counts_0.head(2)

,ill,dreadfully,mary,surprise,hatter,longer,right,tell,bad,dish,...,here,deep,improve,yes,text_sentence,text_source,punc_count,adj_count,verb_count,conj_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,10,3,13,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,7,7,11,0


In [105]:
word_counts_0 = word_counts_0.drop('adj_count', 1)

In [106]:
word_counts_0.head(2)

,ill,dreadfully,mary,surprise,hatter,longer,right,tell,bad,dish,...,heart,here,deep,improve,yes,text_sentence,text_source,punc_count,verb_count,conj_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,10,13,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,7,11,0


Try the logistics regression again with L2 regularization parameter

In [96]:
Y = word_counts_0['text_source']
X = np.array(word_counts_0.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [97]:
# Logistics regression with out regularization parameter
log = LogisticRegression()
train = log.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', log.score(X_train, y_train))
print('\nTest set score:', log.score(X_test, y_test))

(3190, 778) (3190,)
Training set score: 0.9432601880877743

Test set score: 0.9172932330827067


The result has only increased by less than one percent from the original score above. 

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [13]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [112]:
stories = gutenberg.raw('bryant-stories.txt')

In [113]:
# visula inspection
print(stories[:200])

[Stories to Tell to Children by Sara Cone Bryant 1918] 


TWO LITTLE RIDDLES IN RHYME


     There's a garden that I ken,
     Full of little gentlemen;
     Little caps of blue they wear,
  


In [114]:
# Load and process the same way

stories = re.sub(r'VOLUME \w+', '', stories)
stories = re.sub(r'CHAPTER \w+', '', stories)
stories = re.sub(r'Book \w+', '', stories)
stories = text_cleaner(stories)
print(stories[:200])

# Parse the data
stories_doc = nlp(stories)

TWO LITTLE RIDDLES IN RHYME There's a garden that I ken, Full of little gentlemen; Little caps of blue they wear, And green ribbons, very fair. (Flax.) From house to house he goes, A messenger small a


In [115]:
# Group sentences and make the parse same length as Alice 
stories_sents = [[sent, 'Bryant'] for sent in stories_doc.sents]
stories_sents = stories_sents[0:len(alice_sents)]

In [116]:
# Create bow data frame for hamlet 
stories_sentences = pd.DataFrame(stories_sents)
stories_bow = bow_features(stories_sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500


We now add the same features on the new data set as the ones we used in alice data set 

In [117]:
text_sentence = hamlet_bow.text_sentence
# punctuation mark count in each sentence
punc_count = []
for sent in text_sentence:
    punc = 0
    for token in sent:
        if token.pos_ == 'PUNCT':
            punc +=1
    punc_count.append(punc)

stories_bow['punc_count'] = punc_count

# Adjuctive count in each sentence 
#adj_count = []
#for sent in text_sentence:
#    adj = 0
#    for token in sent:
#        if token.pos_ == 'ADJ':
#            adj +=1
#    adj_count.append(adj)

## add as a column
#stories_bow['adj_count'] = adj_count

# verb count in each sentence
verb_count = []
for sent in text_sentence:
    verb = 0
    for token in sent:
        if token.pos_ == 'VERB':
            verb +=1
    verb_count.append(verb)
    
# add as a column
stories_bow['verb_count'] = verb_count

# conjunction count 
conj_count = []
for sent in text_sentence:
    conj = 0
    for token in sent:
        if token.pos_ == 'CONJ':
            conj +=1
    conj_count.append(conj)

stories_bow['conj_count'] = conj_count

In [118]:
# Confirm changes 
stories_bow.head()

,ill,dreadfully,mary,surprise,hatter,longer,right,tell,bad,dish,...,heart,here,deep,improve,yes,text_sentence,text_source,punc_count,verb_count,conj_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(TWO, LITTLE, RIDDLES, IN, RHYME)",Bryant,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(There, 's, a, garden, that, I, ken, ,, Full, ...",Bryant,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"((, Flax, ., ))",Bryant,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(From, house, to, house, he, goes, ,, A, messe...",Bryant,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"((, The, path, ., ))",Bryant,1,1,0


In [108]:
hamlet_bow = hamlet_bow.drop('adj_count', 1)

** Modeling with Logistics regression** 

In [123]:
X_stories = stories_bow.drop(['text_sentence', 'text_source'], 1)
Y_stories = stories_bow.text_source

alice_count = word_counts_0[word_counts_0.text_source == 'Carroll']
X_alice = alice_count.drop(['text_sentence', 'text_source'], 1)
Y_alice = alice_count.text_source

X = pd.concat([X_stories, X_alice], 0)
Y = pd.concat([Y_stories, Y_alice], 0)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

log = LogisticRegression()
train = log.fit(X_train, y_train)
print('train set: {}'.format(log.score(X_train, y_train)))
print('test set: {}'.format(log.score(X_test, y_test)))
log_stories_predicted = log.predict(X_test)
pd.crosstab(y_test, log_stories_predicted)

train set: 0.8976023976023976
test set: 0.8510479041916168


col_0,Bryant,Carroll
text_source,,
Bryant,609,80
Carroll,119,528


The model is able to identify Carrolll vs Bryant reasonably well. It might be possible to increase the accuracy by using more data and creating more features. Other models such as Gradient boost might also increase the accuracy but that is very slow to run. Testing the model with other materials will also be necessary before concluding that it is able to predict other works.  I have used only 500 most common words to reduce the run time. 